# Линейная регрессия с Statsmodels. Вывод результатов подгонка регрессии

In [ ]:
import numpy as np
import pandas as pd

import statsmodels.formula.api as smf

from statsmodels.iolib.summary2 import summary_col, summary_params # вывод результатов 

In [ ]:
df = pd.read_csv('sleep75.csv')
df.shape

## Пример: t-тест (sleep equation)

На датасете/датафрейме `df` подгоним регрессия `sleep` на `totwrk, age`. Спецификация (как уравнение)
$$
	sleep=\beta_0+\beta_1 totwrk+\beta_2 age+u
$$

In [ ]:
# инициализация/спецификация модели через формулу
# создаём объект класса OLS
mod = smf.ols(formula='sleep~totwrk+age', data=df)

# подгонка специфицированной модели на данных с неробастной ковариационной матрицей для коэффициентов
# создаём объект класса RegressionResults
res = mod.fit()

# выведем результаты t-тестя для коэффициентов с округлением
# параметр alpha отвечает за доверительный интервал, xnames - за названия регрессоров (если нужно)
# т.к. создаётся таблица как датафрейм, то её можно сохранить как файл обычными методами
summary_params(res, alpha=0.05, xname=['const', 'занятость', 'возраст']) .round(3)

## Пример: вывод результатов подгонки нескольких регрессоров
На датасете/датафрейме `df` подгоним несколько регрессий `sleep` на 
- `totwrk, age` 
- `totwrl`, `age`, `age^2`
- `totwrk`, `male`, `smsa`, `south`
- `totwrk`, `male`, `smsa`, `south`, `log(hrwage)`

и выведем результаты подгонки в виде одной таблицы

In [ ]:
# Подгоним три модели
res1 = smf.ols(formula='sleep~totwrk+age', data=df).fit()

res2 = smf.ols(formula='sleep~totwrk+age+I(age**2)', data=df).fit()

res3 = smf.ols(formula='sleep~totwrk+male+smsa+south', data=df).fit()

res4 = smf.ols(formula='sleep~totwrk+male+smsa+south+np.log(hrwage)', data=df).fit()

# выведем результаты трёх моделей в одной таблице
# model_names -  имена моделей
# stars - стандартные коды значимости для коэффициентов
# float_format - формат вывода числовых значений, например сколько десятичных знаков
summary_col(results=[res1, res2, res3, res4], model_names=['Модель 1', 'Модель 2', 'Модель 3', 'Модель 4'], 
            stars=True, float_format='%.3f')

In [ ]:
# изменим порядок вывода регрессоров, параметр regressor_order
summary_col(results=[res1, res2, res3, res4], model_names=['Модель 1', 'Модель 2', 'Модель 3', 'Модель 4'], 
            stars=True, float_format='%.3f', 
            regressor_order=['Intercept', 'totwrk', 'age', 'I(age ** 2)', 'male', 'smsa', 'south', 'np.log(hrwage)'])

## Пример: настройка вывода результатов подгонки нескольких регрессоров
На датасете/датафрейме `df` подгоним несколько регрессий `sleep` на 
- `totwrk, age` 
- `totwrl`, `age`, `age^2`
- `totwrk`, `male`, `smsa`, `south`
- `totwrk`, `male`, `smsa`, `south`, `log(hrwage)`

и выведем результаты подгонки в виде одной таблицы.

В таблице можно настроить вывод свойства объекта класса [RegressionResults](https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.RegressionResults.html#statsmodels.regression.linear_model.RegressionResults) (подогнанная модель). Например:
- число наблюдений `nobs`
- F-статистика `fvalue` и её P-значение `f_pvalue`
- Информационные критерии `aic`, `bic`
- $R^2$ и $R^2_{adj}$ `rsquared`, `rsquared_adj`

Причём это модно сделать отдельно для каждой модели

**Важно**: будем делать в формате словаря с использованием lambda - функций

### Добавление  числа наблюдений, F-статистики и её P-значения

In [ ]:
# Подгоним три модели
res1 = smf.ols(formula='sleep~totwrk+age', data=df).fit()

res2 = smf.ols(formula='sleep~totwrk+age+I(age**2)', data=df).fit()

res3 = smf.ols(formula='sleep~totwrk+male+smsa+south', data=df).fit()

res4 = smf.ols(formula='sleep~totwrk+male+smsa+south+np.log(hrwage)', data=df).fit()

# Имена моделей
mod_names = ['Модель 1', 'Модель 2', 'Модель 3', 'Модель 4']
# порядок регрессоров в таблице
reg_order = ['Intercept', 'totwrk', 'age', 'I(age ** 2)', 'male', 'smsa', 'south', 'np.log(hrwage)']
# словарь с тем, что добавим
info_to_add = {'n.obs': lambda x: x.nobs, 'F-stat': lambda x: x.fvalue, 'P(F-stat)': lambda x: x.f_pvalue }

summary_col(results=[res1, res2, res3, res4], model_names=mod_names, stars=True, float_format='%.3f', 
            regressor_order=reg_order, info_dict=info_to_add)

In [ ]:
output = summary_col(results=[res1, res2, res3, res4], model_names=mod_names, stars=True, float_format='%.3f', regressor_order=reg_order, info_dict=info_to_add)

with open('regressions_output.txt', 'w') as f:
	f.write(output.as_text())